In [7]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from scipy.sparse import csr_matrix, hstack

In [9]:
PATH = '../../data/'

print('reading data')

train = pd.read_csv(PATH + 'cleaned_train.csv')
test = pd.read_csv(PATH + 'cleaned_test.csv')

print('data cleaning')

tok = TweetTokenizer()
lem = WordNetLemmatizer()
stopword = set(stopwords.words("english"))

def clean(comment):
    comment = re.sub(r'-', ' ', comment)
    text = tok.tokenize(comment)
    text = [w for w in text if not w in stopword]
    text = [word for word in text if not re.match(r'http:\/\/.*', word)]
    text = [lem.lemmatize(word, 'n') for word in text]
    text = ' '.join(text)
    if text == '': text = 'na'
    return text

train['comment_text_cleaned'] = train['comment_text_cleaned'].apply(lambda x: clean(x))
test['comment_text_cleaned'] = test['comment_text_cleaned'].apply(lambda x: clean(x))

print('calculations')

train_sentence = train['comment_text']
test_sentence = test['comment_text']

def f(x):
    x = tok.tokenize(x)
    return len(x)

text = pd.concat([train_sentence, test_sentence])

print(train.shape)
print(test.shape)

reading data
data cleaning
calculations
(159571, 27)
(153164, 21)


In [13]:
# CountVectorizer(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None,\
#                 lowercase=True, preprocessor=None, tokenizer=None, stop_words=None,\
#                 token_pattern=’(?u)\b\w\w+\b’, ngram_range=(1, 1), analyzer=’word’, max_df=1.0,\
#                 min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class ‘numpy.int64’>)

# TfidfVectorizer(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None,\
#                 lowercase=True, preprocessor=None, tokenizer=None, analyzer=’word’, stop_words=None,\
#                 token_pattern=’(?u)\b\w\w+\b’, ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None,\
#                 vocabulary=None, binary=False, dtype=<class ‘numpy.int64’>, norm=’l2’, use_idf=True,\
#                 smooth_idf=True, sublinear_tf=False)

print('getting tfidf')
char_vectorizer = TfidfVectorizer(ngram_range=(1,3), stop_words='english', max_df=0.9, min_df=100,\
                                  strip_accents='unicode', use_idf=1, smooth_idf=1, sublinear_tf=1,
                                  max_features=20000, analyzer='char')
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,5), stop_words='english', max_df=0.9, min_df=100,\
                                    strip_accents='unicode', use_idf=1, smooth_idf=1, sublinear_tf=1,
                                    max_features=35000)
print('fitting char')
char_vectorizer.fit(text.values)
# print('fitting phrase')
# phrase_vectorizer.fit(text.values)
# print('transforming train char')
# train_char = char_vectorizer.transform(train['comment_text_cleaned'].values)
# print('transforming train phrase')
# train_phrase = phrase_vectorizer.transform(train['comment_text_cleaned'].values)
# print('transforming test char')
# test_char = char_vectorizer.transform(test['comment_text_cleaned'].values)
# print('transforming test phrase')
# test_phrase = phrase_vectorizer.transform(test['comment_text_cleaned'].values)

# train_tfidf = hstack((train_char, train_phrase), format='csr')
# test_tfidf = hstack((test_char, test_phrase), format='csr')

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train_tfidf

getting tfidf
fitting char


<159571x15556 sparse matrix of type '<class 'numpy.float64'>'
	with 3686894 stored elements in Compressed Sparse Row format>

In [14]:
print(char_vectorizer.get_feature_names())

['\n', '\n!', '\n! ', '\n"', '\n""', "\n'", '\n(', '\n-', '\n.', '\n..', '\n1', '\n1.', '\n2', '\n2.', '\n3', '\n3.', '\n4', '\n5', '\n6', '\n7', '\n8', '\n9', '\n[', '\na', '\na ', '\nac', '\nad', '\nag', '\nal', '\nan', '\nar', '\nas', '\nat', '\nb', '\nba', '\nbe', '\nbo', '\nbu', '\nby', '\nc', '\nca', '\nch', '\nco', '\nd', '\nda', '\nde', '\ndi', '\ndo', '\ne', '\ned', '\nev', '\nex', '\nf', '\nfa', '\nfi', '\nfo', '\nfr', '\nfu', '\ng', '\nge', '\ngo', '\ngr', '\nh', '\nha', '\nhe', '\nhi', '\nho', '\nht', '\ni', '\ni ', "\ni'", '\nif', '\nin', '\nis', '\nit', '\nj', '\nju', '\nk', '\nl', '\nla', '\nle', '\nli', '\nlo', '\nm', '\nma', '\nme', '\nmi', '\nmo', '\nmy', '\nn', '\nna', '\nne', '\nno', '\no', '\nof', '\noh', '\nok', '\non', '\nor', '\np', '\npa', '\npe', '\npl', '\npo', '\npr', '\nr', '\nre', '\ns', '\nsa', '\nse', '\nsh', '\nsi', '\nso', '\nst', '\nsu', '\nt', '\nta', '\nte', '\nth', '\nto', '\ntr', '\ntu', '\nu', '\nun', '\nus', '\nv', '\nw', '\nwa', '\nwe', '\nwh',

In [4]:
# print('combine featrues')

# other_feature_cols = ['word_count', 'cleaned_word_count', 'unique_word_count', 'cleaned_unique_word_count',\
#                       'question_marks', 'consecutive_question_marks', 'exclamation_marks',\
#                       'consecutive_exclamation_marks', 'uppercase_letters', 'ellipsis', 'period',\
#                       'parentheses_pair', 'special_symbol', 'sentence', 'upper_word_ratio', 'unique_word_ratio',\
#                       'mark_count_ratio']

other_feature_cols = ['upper_word_ratio']

print('getting train features')
train_features = [train_tfidf, csr_matrix(train[other_feature_cols].values)]
# train_features = [train_tfidf, []]

print('getting test features')
test_features = [test_tfidf, csr_matrix(test[other_feature_cols].values)]
# test_features = [test_tfidf, []]

train_features

getting train features
getting test features


[<159571x15556 sparse matrix of type '<class 'numpy.float64'>'
 	with 3686894 stored elements in Compressed Sparse Row format>,
 <159571x1 sparse matrix of type '<class 'numpy.float64'>'
 	with 153358 stored elements in Compressed Sparse Row format>]

In [5]:
def pr(y_i, y, train_features):
    p = train_features[y==y_i].sum(0)
    return (p + 1) / ((y == y_i).sum() + 1)

def get_nblogreg_model(label_cols, train_features, train, test_features):
    preds = np.zeros((test.shape[0], len(label_cols)))
    train_preds = np.zeros((train.shape[0], len(label_cols)))
    for i, j in enumerate(label_cols):
        print('fit', j)
        y = train[j].values
        r = np.log(pr(1, y, train_features[0]) / pr(0, y, train_features[0]))
        model = LogisticRegression(C=4, dual=True)
#         x_nb = hstack((train_features[0].multiply(r), train_features[1]), format='csr')
        x_nb = hstack((train_features[0], train_features[1]), format='csr')
        model.fit(x_nb, y)
#         preds[:, i] = model.predict_proba(hstack((test_features[0].multiply(r), test_features[1]), format='csr'))[:, 1]
        preds[:, i] = model.predict_proba(hstack((test_features[0], test_features[1]), format='csr'))[:, 1]
        train_preds[:, i] = model.predict_proba(x_nb)[:, 1]
        print('accuracy is {}'.format(roc_auc_score(y, train_preds[:, i])))
    return preds, train_preds

def save(model_name, y_test, label_cols, path, is_train=False):
    if is_train:
        submission = pd.read_csv(path + 'sample_train.csv')
        file_name = 'train_' + model_name
    else:
        submission = pd.read_csv(path + 'sample_submission.csv')
        file_name = model_name
    submission[label_cols] = y_test
    submission.to_csv(path + model_name + '/' + file_name + '.csv', index=False)
    
print('done')

done


In [6]:
print('predicting')
n = 1
print('the {}-th step'.format(0))
y_test, y_train = get_nblogreg_model(label_cols, train_features, train, test_features)
print('\n\n')
for i in range(n - 1):
    print('the {}-th step'.format(i + 1))
    tmp1, tmp2 = get_nblogreg_model(label_cols, train_features, train, test_features)
    y_test += tmp1
    y_train += tmp2
    print('\n\n')
    
y_test /= n
y_train /= n

print('total score is {}'.format(roc_auc_score(train[label_cols], y_train)))

print('saving files')
model_name = 'nblogreg'
save(model_name, y_test, label_cols, PATH)
save(model_name, y_train, label_cols, PATH, True)

print('done')

predicting
the 0-th step
fit toxic
accuracy is 0.9830381045482812
fit severe_toxic
accuracy is 0.9940975657903197
fit obscene
accuracy is 0.9916186396304425
fit threat
accuracy is 0.9971766073405606
fit insult
accuracy is 0.9865248031390055
fit identity_hate
accuracy is 0.9936653854774768



total score is 0.9910201843210144
saving files
done
